In [1]:
from ngraph.scenario import Scenario
from ngraph.traffic_demand import TrafficDemand
from ngraph.traffic_manager import TrafficManager
from ngraph.lib.flow_policy import FlowPolicyConfig, FlowPolicy, FlowPlacement
from ngraph.lib.algorithms.base import PathAlg, EdgeSelect

In [2]:
scenario_yaml = """
blueprints:
  brick_2tier:
    groups:
      t1:
        node_count: 8
        name_template: t1-{node_num}
      t2:
        node_count: 8
        name_template: t2-{node_num}

    adjacency:
      - source: /t1
        target: /t2
        pattern: mesh
        link_params:
          capacity: 2
          cost: 1

  3tier_clos:
    groups:
      b1:
        use_blueprint: brick_2tier
      b2:
        use_blueprint: brick_2tier
      spine:
        node_count: 64
        name_template: t3-{node_num}

    adjacency:
      - source: b1/t2
        target: spine
        pattern: one_to_one
        link_params:
          capacity: 2
          cost: 1
      - source: b2/t2
        target: spine
        pattern: one_to_one
        link_params:
          capacity: 2
          cost: 1

network:
  name: "3tier_clos_network"
  version: 1.0

  groups:
    my_clos1:
      use_blueprint: 3tier_clos

    my_clos2:
      use_blueprint: 3tier_clos

  adjacency:
    - source: my_clos1/spine
      target: my_clos2/spine
      pattern: one_to_one
      link_params:
        capacity: 2
        cost: 1
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network

In [3]:
network.max_flow(
    source_path=r"my_clos1.*(b[0-9]*)/t1",
    sink_path=r"my_clos2.*(b[0-9]*)/t1",
    mode="combine",
    shortest_path=True,
)

{('b1|b2', 'b1|b2'): 128.0}

In [ ]:
d = TrafficDemand(
    source_path=r"my_clos1.*(b[0-9]*)/t1",
    sink_path=r"my_clos2.*(b[0-9])/t1",
    demand=10,
    mode="full_mesh",
    flow_policy_config=FlowPolicyConfig.SHORTEST_PATHS_ECMP,
)
demands = [d]
tm = TrafficManager(
    network=network,
    traffic_demands=demands,
)
tm.build_graph()
tm.expand_demands()
tm.place_all_demands(placement_rounds=50)

0.0

In [5]:
d

TrafficDemand(source_path='my_clos1.*(b[0-9]*)/t1', sink_path='my_clos2.*(b[0-9])/t1', priority=0, demand=10, demand_placed=0.0, flow_policy_config=<FlowPolicyConfig.TE_ECMP_16_LSP: 5>, flow_policy=None, mode='full_mesh', attrs={}, id='my_clos1.*(b[0-9]*)/t1|my_clos2.*(b[0-9])/t1|S6weVhAgQMCerTqMbTnMVw')

In [9]:
tm.demands

[Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-1', volume=0.0390625, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x10c0bbcb0>, placed_demand=0.0),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-2', volume=0.0390625, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x104521a90>, placed_demand=0.0),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-3', volume=0.0390625, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x104521310>, placed_demand=0.0),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-4', volume=0.0390625, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x104762190>, placed_demand=0.0),
 Demand(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-5', volume=0.0390625, demand_class=0, flow_policy=<ngraph.lib.flow_policy.FlowPolicy object at 0x10c11df30>, placed_demand=0.0),
 Demand(src_nod

In [7]:
vars(dmd.flow_policy)

{'path_alg': <PathAlg.SPF: 1>,
 'flow_placement': <FlowPlacement.EQUAL_BALANCED: 2>,
 'edge_select': <EdgeSelect.SINGLE_MIN_COST_WITH_CAP_REMAINING_LOAD_FACTORED: 6>,
 'multipath': False,
 'min_flow_count': 16,
 'max_flow_count': 16,
 'max_path_cost': None,
 'max_path_cost_factor': None,
 'static_paths': None,
 'edge_select_func': None,
 'edge_select_value': None,
 'reoptimize_flows_on_each_placement': True,
 'flows': {FlowIndex(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-1', flow_class=0, flow_id=0): <ngraph.lib.flow.Flow at 0x10c424050>,
  FlowIndex(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-1', flow_class=0, flow_id=1): <ngraph.lib.flow.Flow at 0x10c06fb10>,
  FlowIndex(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-1', flow_class=0, flow_id=2): <ngraph.lib.flow.Flow at 0x10c06f9d0>,
  FlowIndex(src_node='my_clos1/b1/t1/t1-1', dst_node='my_clos2/b1/t1/t1-1', flow_class=0, flow_id=3): <ngraph.lib.flow.Flow at 0x10c11e8b0>,
  FlowIndex(src